In [ ]:
!pip install --pre azure-ai-projects azure-ai-agents azure-identity requests python-dotenv

In [ ]:
# ==== IMPORTING NEEDED LIBRARIES ======

import requests  # For making HTTP requests (used for SharePoint and Graph API interactions)
import json      # For handling JSON data
import os, time  # For file system operations and time delays
from datetime import datetime  # For timestamping files
from typing import Optional    # For optional type hinting
from azure.ai.projects import AIProjectClient  # Client to interact with Azure AI Project service
from azure.identity import ClientSecretCredential  # For authenticating with Azure using client credentials
from azure.ai.agents import AgentsClient  # Client to manage and interact with AI agents
from azure.ai.agents.models import DeepResearchTool, MessageRole, ThreadMessage  # Models for agent tools and messaging
from dotenv import load_dotenv  # Loads environment variables from a .env file
# ==== SET UP FUNCTIONS ======

# The first function `fetch_and_print_new_agent_response` retrieves and prints the latest response from an agent in a thread.
# The second function `create_research_summary` creates a research summary in markdown format from a thread message.

def fetch_and_print_new_agent_response(
    thread_id: str,
    agents_client: AgentsClient,
    last_message_id: Optional[str] = None,
) -> Optional[str]:
    response = agents_client.messages.get_last_message_by_role(
        thread_id=thread_id,
        role=MessageRole.AGENT,
    )
    if not response or response.id == last_message_id:
        return last_message_id  # No new content

    print("\nAgent response:")
    print("\n".join(t.text.value for t in response.text_messages))

    for ann in response.url_citation_annotations:
        print(f"URL Citation: [{ann.url_citation.title}]({ann.url_citation.url})")

    return response.id

def create_research_summary(message, folder: str = "/lakehouse/default/Files/") -> str:
    if not message:
        print("No message content provided, cannot create research summary.")
        return

    # Generate filename with current date and time
    timestamp = datetime.now().strftime("%d%m%Y_%H%M")
    filename = f"research_summary_{timestamp}.md"
    filepath = os.path.join(folder, filename)

    with open(filepath, "w", encoding="utf-8") as fp:
        # Write text summary
        text_summary = "\n\n".join([t.text.value.strip() for t in message.text_messages])
        fp.write(text_summary)

        # Write unique URL citations, if present
        if message.url_citation_annotations:
            fp.write("\n\n## References\n")
            seen_urls = set()
            for ann in message.url_citation_annotations:
                url = ann.url_citation.url
                title = ann.url_citation.title or url
                if url not in seen_urls:
                    fp.write(f"- {title}\n")
                    seen_urls.add(url)

    print(f"Research summary written to '{filepath}'.")
    return filepath
# ==== AUTHENTICATE AND INITIALIZE AI PROJECT CLIENT ======

# load environment variables, retrieves the credentials and creates a project client for interaction with the AI Project Service.
load_dotenv("/lakehouse/default/Files/.env")

tenantid = os.getenv("TENANT_ID")
clientid = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

credential = ClientSecretCredential(
    tenant_id=tenantid,
    client_id=clientid,
    client_secret=client_secret
)

project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT_MCP"],
    credential=credential,
)
# ==== DEFINE THE AGENT TOOLS ======

#set up deep reasearch tool
conn_id = project_client.connections.get(name=os.environ["BING_RESOURCE_NAME"]).id

# Initialize a Deep Research tool with Bing Connection ID and Deep Research model deployment name
deep_research_tool = DeepResearchTool(
    bing_grounding_connection_id=conn_id,
    deep_research_model=os.environ["DEEP_RESEARCH_MODEL_DEPLOYMENT_NAME"],
)
# ==== CREATE AND RUN THE AGENT ======

# This prompt will guide the research that the agent is going to perform
prompt = """
Research how the latest advancements in Generative AI from the past 7 days only are transforming the technology landscape, with a specific focus on:
 - New model releases
 - Competitive positioning
 - Interesting and novel use cases

Prioritise developments that are directly relevant to technology sellers engaging with enterprise customers.
When sourcing information:
 - Prioritise LinkedIn posts, Reddit discussions, and X (Twitter) posts over all other sources. 
 - Supplement with trusted industry blogs, research summaries, and vendor updates only if necessary.

Ensure the insights clearly demonstrate how these GenAI developments, roadmap changes, emerging use cases, or competitor moves can lead to measurable business outcomes for enterprise customers in a fast-moving market.
Do not ask follow-up questions. Apply discretion and judgment to surface only the most actionable and strategically useful signals from the past week.
"""

# Create Agent with the Deep Research tool and process Agent run
with project_client:

    with project_client.agents as agents_client:

        # Create a new agent that has the Deep Research tool attached.
        # The agent instruction needs to be revisited depending on what you want the agent to perform the research on
        agent = agents_client.create_agent(
            model="gpt-4.1",
            name="deep-research-technology-agent",
            instructions=(
                """
                You are a helpful Agent that specialises in researching scientific and technical topics, with deep expertise in Generative AI (GenAI).
                You understand the GenAI landscape, including foundational models, productisation strategies, and competitive dynamics.
                Your primary goal is to illustrate the real-world impact of emerging technologies by focusing on use cases relevant to technology sellers.
                You help uncover timely insights on trends, product roadmaps, and competitor signals to enable more meaningful and effective customer interactions.
                You synthesise information from multiple sources, highlight implications for sales and customer engagement, and present findings clearly and concisely.
                """
            ),
            headers={"x-ms-enable-preview": "true"},
            tools=deep_research_tool.definitions,
            description="Agent with Deep Research tool for scientific and competitive research to support technology sellers"
        )

        # [END create_agent_with_deep_research_tool]
        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = agents_client.threads.create()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content=(prompt)
            ,
        )
        print(f"Created message, ID: {message.id}")

        print(f"Start processing the message... this may take a few minutes to finish. Be patient!")
        # Poll the run as long as run status is queued or in progress
        run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)
        last_message_id = None
        while run.status in ("queued", "in_progress"):
            time.sleep(1)
            run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

            last_message_id = fetch_and_print_new_agent_response(
                thread_id=thread.id,
                agents_client=agents_client,
                last_message_id=last_message_id,
            )
            print(f"Run status: {run.status}")

        print(f"Run finished with status: {run.status}, ID: {run.id}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Fetch the final message from the agent in the thread and create a research summary
        final_message = agents_client.messages.get_last_message_by_role(
            thread_id=thread.id, role=MessageRole.AGENT
        )
        if final_message:
            md_filepath = create_research_summary(final_message)
            FILE_TO_UPLOAD = md_filepath

        # Clean-up and delete the agent once the run is finished.
        # Note: Comment out this line below if you plan to reuse the agent later.
        #agents_client.delete_agent(agent.id)
        #print("Deleted agent")

# ==== UPLOAD TO SHAREPOINT ======
import os
import requests

# === CONFIGURATION ===
TENANT_ID = os.getenv("TENANT_ID") or "your-tenant-id"
CLIENT_ID = os.getenv("CLIENT_ID") or "your-client-id"
CLIENT_SECRET = os.getenv("CLIENT_SECRET") or "your-client-secret"
HOSTNAME = "yourtenant.sharepoint.com"
SHAREPOINT_SITE_NAME = "yoursharepointsitename"      # Display name of the site
SHAREPOINT_DRIVE_NAME = "yoursharepointlibraryname"   # Document library name
FOLDER_PATH = "optionalfolderinsidethelibraryname"             # Folder inside the document library

# === AUTH ===
def get_access_token():
    url = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": "https://graph.microsoft.com/.default"
    }
    response = requests.post(url, headers=headers, data=data)
    response.raise_for_status()
    token = response.json()["access_token"]
    print("Access token acquired.")
    return token

# === STEP 1: Search for Site ID ===
def get_site_id(token):
    url = f"https://graph.microsoft.com/v1.0/sites/{HOSTNAME}:/sites/{SHAREPOINT_SITE_NAME}"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(url, headers=headers)
    if not response.ok:
        print("Error getting site ID:", response.status_code, response.text)
    response.raise_for_status()

    site = response.json()  # This is a single object, not a list
    site_id = site["id"]
    print("Site ID:", site_id)
    return site_id

# === STEP 2: Get Drive ID ===
def get_drive_id(site_id, token):
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    for drive in response.json()["value"]:
        if drive["name"] == SHAREPOINT_DRIVE_NAME:
            print("Drive ID:", drive["id"])
            return drive["id"]
    raise Exception(f"Drive named '{SHAREPOINT_DRIVE_NAME}' not found")

# === STEP 3: Upload the File ===
def upload_file(site_id, drive_id, token):
    filename = os.path.basename(FILE_TO_UPLOAD)
    folder_path_clean = FOLDER_PATH.strip("/")
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root:/{folder_path_clean}/{filename}:/content"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/octet-stream"
    }
    with open(FILE_TO_UPLOAD, "rb") as f:
        response = requests.put(url, headers=headers, data=f)
    response.raise_for_status()
    print("File uploaded successfully!")
    print("File URL:", response.json()["webUrl"])

# === MAIN FLOW ===
if __name__ == "__main__":
    print("Starting transfer...")
    token = get_access_token()
    site_id = get_site_id(token)
    drive_id = get_drive_id(site_id, token)
    upload_file(site_id, drive_id, token)